In [ ]:
import os
import json
import numpy as np
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.circuits.cp_fraction import cp_fraction
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
import math as mt

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################

detailed_filename = "benchmark_results_FM_CP_fast.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_fast.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2 + i

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    network = QuantumNetwork(qpu_info)
    
    # Sweep the fraction parameter from 0.1 to 0.9
    for fraction in np.arange(0.3, 0.9, 0.2):
        
        # Collect data for computing means across 10 iterations
        iteration_data = []
        
        for iteration in range(5):

            # -------------------------
            # 1. Define circuit
            # -------------------------
            circuit = cp_fraction(num_qubits, num_qubits, fraction)
            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
            depth = base_graph.depth
            num_levels = int(np.ceil(mt.log2(depth))) + 1
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. FM partitioning
            # -------------------------
            graph_list = [base_graph]
            # Set a mapping list for just one level since no coarsening
            mapping_list = [{i : set([i]) for i in range(depth)}]

            assignment_list_f, cost_list_f, time_list_f = multilevel_FM(graph_list,
                                                                        mapping_list,
                                                                        initial_assignment,
                                                                        qpu_info,
                                                                        limit = num_qubits*0.125,
                                                                        pass_list= [5*num_levels],
                                                                        stochastic=True,
                                                                        lock_nodes=False)
            total_time_f = sum(time_list_f)
            min_cost_f = min(cost_list_f)
            # -------------------------
            # 3. MLFM-W partitioning
            # -------------------------

            assignment_list_w, cost_list_w, time_list_w = MLFM_window(base_graph, 
                num_levels=num_levels,
                initial_assignment=initial_assignment,  
                qpu_info= qpu_info, 
                limit = num_qubits*0.125, 
                pass_list= [5]*(num_levels+2), 
                stochastic=True, 
                lock_nodes=False,
                log = False,
                add_initial = False,
                costs = None)
            
            total_time_w = sum(time_list_w)
            min_cost_w = min(cost_list_w)

            
            # -------------------------
            # 4. MLFM-B partitioning
            # -------------------------
            assignment_list_b, cost_list_b, time_list_b = MLFM_blocks(base_graph,
                                            num_levels = num_levels,
                                            initial_assignment = initial_assignment,  
                                            qpu_info = qpu_info, 
                                            limit = num_qubits*0.125, 
                                            pass_list= [5]*(num_levels+2), 
                                            stochastic = True, 
                                            lock_nodes = False,
                                            log = False,
                                            add_initial = False,
                                            costs = None)
            
            total_time_b = sum(time_list_b)
            min_cost_b = min(cost_list_b)
            
            # -------------------------
            # 5. MLFM-R partitioning
            # -------------------------
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits*0.125, 
                                        pass_list= [5]*(num_levels+2), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": float(fraction),
                "iteration": iteration,
                "f_cost": min_cost_f,
                "w_cost": min_cost_w,
                "b_cost": min_cost_b,
                "r_cost":  min_cost_r,
                "time_f": total_time_f,
                "time_w": total_time_w,
                "time_b": total_time_b,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------
        f_cost_list = [x["f_cost"] for x in iteration_data]
        w_cost_list = [x["w_cost"] for x in iteration_data]
        b_cost_list = [x["b_cost"] for x in iteration_data]
        r_cost_list = [x["r_cost"] for x in iteration_data]
        
        f_time_list = [x["time_f"] for x in iteration_data]
        w_time_list = [x["time_w"] for x in iteration_data]
        b_time_list = [x["time_b"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_f_cost = float(np.mean(f_cost_list))
        mean_w_cost = float(np.mean(w_cost_list))
        mean_b_cost = float(np.mean(b_cost_list))
        mean_r_cost = float(np.mean(r_cost_list))
        
        mean_f_time = float(np.mean(f_time_list))
        mean_w_time = float(np.mean(w_time_list))
        mean_b_time = float(np.mean(b_time_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, fraction={fraction}")
        print("Mean Costs:")
        print(f"  `Fine`:    {mean_f_cost:.3f}")
        print(f"  Window: {mean_w_cost:.3f}")
        print(f"  Block: {mean_b_cost:.3f}")
        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")
        print(f"  Fine:    {mean_f_time:.3f}")
        print(f"  Window: {mean_w_time:.3f}")
        print(f"  Block: {mean_b_time:.3f}")
        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction": float(fraction),
            "mean_f_cost": mean_f_cost,
            "mean_w_cost": mean_w_cost,
            "mean_b_cost": mean_b_cost,
            "mean_r_cost": mean_r_cost,
            "mean_f_time": mean_f_time,
            "mean_w_time": mean_w_time,
            "mean_b_time": mean_b_time,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.3
Mean Costs:
  `Fine`:    11.800
  Window: 5.400
  Block: 5.600
  Recursive:5.800
Mean Times (s):
  Fine:    0.041
  Window: 0.028
  Block: 0.029
  Recursive:0.034
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.5
Mean Costs:
  `Fine`:    19.600
  Window: 12.600
  Block: 13.400
  Recursive:12.200
Mean Times (s):
  Fine:    0.044
  Window: 0.033
  Block: 0.044
  Recursive:0.037
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.7
Mean Costs:
  `Fine`:    24.800
  Window: 15.600
  Block: 15.600
  Recursive:15.800
Mean Times (s):
  Fine:    0.049
  Window: 0.037
  Block: 0.036
  Recursive:0.040
Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.9
Mean Costs:
  `Fine`:    18.600
  Window: 13.800
  Block: 13.800
  Recursive:13.000
Mean Times (s):
  Fine:    0.061
  Window: 0.045
  Block: 0.045
  Recursive:0.045
Finished 10 iterations for:
  # Qubits: 24,

KeyboardInterrupt: 

In [1]:
import os
import json
import numpy as np
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.circuits.cp_fraction import cp_fraction
from disqco.parti.FM.FM_main import *
from disqco.parti.FM.multilevel_FM import *
import math as mt

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################

detailed_filename = "benchmark_results_FM_CP_ML_0.5.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FM_CP_ML_0.5.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2 + i

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    network = QuantumNetwork(qpu_info)
    
    # Sweep the fraction parameter from 0.1 to 0.9
    for fraction in [0.5]:
        
        # Collect data for computing means across 10 iterations
        iteration_data = []
        
        for iteration in range(5):

            # -------------------------
            # 1. Define circuit
            # -------------------------
            circuit = cp_fraction(num_qubits, num_qubits, fraction)
            base_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
            depth = base_graph.depth
            num_levels = int(np.ceil(mt.log2(depth))) + 1
            initial_assignment = set_initial_partitions(network,num_qubits, depth)
            
            # -------------------------
            # 2. FM partitioning
            # -------------------------
            graph_list = [base_graph]
            # Set a mapping list for just one level since no coarsening
            mapping_list = [{i : set([i]) for i in range(depth)}]

            assignment_list_f, cost_list_f, time_list_f = multilevel_FM(graph_list,
                                                                        mapping_list,
                                                                        initial_assignment,
                                                                        qpu_info,
                                                                        limit = num_qubits*depth*0.125,
                                                                        pass_list= [10*num_levels],
                                                                        stochastic=True,
                                                                        lock_nodes=False)
            total_time_f = sum(time_list_f)
            min_cost_f = min(cost_list_f)
            # -------------------------
            # 3. MLFM-W partitioning
            # -------------------------

            assignment_list_w, cost_list_w, time_list_w = MLFM_window(base_graph, 
                num_levels=num_levels,
                initial_assignment=initial_assignment,  
                qpu_info= qpu_info, 
                limit = num_qubits, 
                pass_list= [10]*(num_levels+2), 
                stochastic=True, 
                lock_nodes=False,
                log = False,
                add_initial = False,
                costs = None)
            
            total_time_w = sum(time_list_w)
            min_cost_w = min(cost_list_w)

            
            # -------------------------
            # 4. MLFM-B partitioning
            # -------------------------
            assignment_list_b, cost_list_b, time_list_b = MLFM_blocks(base_graph,
                                            num_levels = num_levels,
                                            initial_assignment = initial_assignment,  
                                            qpu_info = qpu_info, 
                                            limit = num_qubits, 
                                            pass_list= [10]*(num_levels+2), 
                                            stochastic = True, 
                                            lock_nodes = False,
                                            log = False,
                                            add_initial = False,
                                            costs = None)
            
            total_time_b = sum(time_list_b)
            min_cost_b = min(cost_list_b)
            
            # -------------------------
            # 5. MLFM-R partitioning
            # -------------------------
            assignment_list_r, cost_list_r, time_list_r = MLFM_recursive(base_graph,
                                        initial_assignment,  
                                        qpu_info, 
                                        limit = num_qubits, 
                                        pass_list= [10]*(num_levels), 
                                        stochastic=True, 
                                        lock_nodes=False,
                                        log = False,
                                        add_initial = False,
                                        costs = None)
            
            total_time_r = sum(time_list_r)
            min_cost_r = min(cost_list_r)

            
            # -------------------------
            # 6. Store iteration-level results
            # -------------------------
            result_entry = {
                "num_qubits": num_qubits,
                "num_partitions": num_partitions,
                "fraction": float(fraction),
                "iteration": iteration,
                "f_cost": min_cost_f,
                "w_cost": min_cost_w,
                "b_cost": min_cost_b,
                "r_cost":  min_cost_r,
                "time_f": total_time_f,
                "time_w": total_time_w,
                "time_b": total_time_b,
                "time_r": total_time_r,
            }
            
            detailed_results.append(result_entry)
            iteration_data.append(result_entry)
            
            # Update detailed JSON right away
            with open(detailed_filename, "w") as f:
                json.dump(detailed_results, f, indent=2)
        
        # ---------------------------------------------------------------------
        # After 10 iterations, compute the means and log them
        # ---------------------------------------------------------------------
        f_cost_list = [x["f_cost"] for x in iteration_data]
        w_cost_list = [x["w_cost"] for x in iteration_data]
        b_cost_list = [x["b_cost"] for x in iteration_data]
        r_cost_list = [x["r_cost"] for x in iteration_data]
        
        f_time_list = [x["time_f"] for x in iteration_data]
        w_time_list = [x["time_w"] for x in iteration_data]
        b_time_list = [x["time_b"] for x in iteration_data]
        r_time_list = [x["time_r"] for x in iteration_data]
        
        mean_f_cost = float(np.mean(f_cost_list))
        mean_w_cost = float(np.mean(w_cost_list))
        mean_b_cost = float(np.mean(b_cost_list))
        mean_r_cost = float(np.mean(r_cost_list))
        
        mean_f_time = float(np.mean(f_time_list))
        mean_w_time = float(np.mean(w_time_list))
        mean_b_time = float(np.mean(b_time_list))
        mean_r_time = float(np.mean(r_time_list))
        
        # Print to console for quick logging
        print("=============================================")
        print(f"Finished 10 iterations for:")
        print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}, fraction={fraction}")
        print("Mean Costs:")
        print(f"  `Fine`:    {mean_f_cost:.3f}")
        print(f"  Window: {mean_w_cost:.3f}")
        print(f"  Block: {mean_b_cost:.3f}")
        print(f"  Recursive:{mean_r_cost:.3f}")
        print("Mean Times (s):")
        print(f"  Fine:    {mean_f_time:.3f}")
        print(f"  Window: {mean_w_time:.3f}")
        print(f"  Block: {mean_b_time:.3f}")
        print(f"  Recursive:{mean_r_time:.3f}")
        print("=============================================")
        
        # Store the aggregated means in a separate JSON
        mean_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "fraction": float(fraction),
            "mean_f_cost": mean_f_cost,
            "mean_w_cost": mean_w_cost,
            "mean_b_cost": mean_b_cost,
            "mean_r_cost": mean_r_cost,
            "mean_f_time": mean_f_time,
            "mean_w_time": mean_w_time,
            "mean_b_time": mean_b_time,
            "mean_r_time": mean_r_time,
        }
        
        mean_results.append(mean_entry)
        
        # Update the means JSON file
        with open(means_filename, "w") as f:
            json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2, fraction=0.5
Mean Costs:
  `Fine`:    13.600
  Window: 12.000
  Block: 12.400
  Recursive:11.000
Mean Times (s):
  Fine:    0.092
  Window: 0.062
  Block: 0.062
  Recursive:0.044
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 3, fraction=0.5
Mean Costs:
  `Fine`:    48.800
  Window: 43.000
  Block: 41.000
  Recursive:40.600
Mean Times (s):
  Fine:    0.436
  Window: 0.252
  Block: 0.253
  Recursive:0.176
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 4, fraction=0.5
Mean Costs:
  `Fine`:    107.400
  Window: 101.400
  Block: 96.200
  Recursive:94.200
Mean Times (s):
  Fine:    1.132
  Window: 0.642
  Block: 0.628
  Recursive:0.425
Finished 10 iterations for:
  # Qubits: 40, # Partitions: 5, fraction=0.5
Mean Costs:
  `Fine`:    196.800
  Window: 198.000
  Block: 177.600
  Recursive:171.400
Mean Times (s):
  Fine:    2.933
  Window: 1.611
  Block: 1.496
  Recursive:0.950
Finished 10 iterations for:
  # Qu